In [2]:
!pip install imbalanced-learn


In [3]:
import pandas as pd

url = "https://raw.githubusercontent.com/AnjulaMehto/Sampling_Assignment/main/Creditcard_data.csv"
df = pd.read_csv(url)

df.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df['Class'].value_counts()


,count
Class,
0,763
1,9


In [5]:
X = df.drop('Class', axis=1)
y = df['Class']


In [6]:
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.combine import SMOTETomek

sampling_methods = {
    "Sampling1": RandomUnderSampler(),
    "Sampling2": RandomOverSampler(),
    "Sampling3": SMOTE(),
    "Sampling4": SMOTETomek(),
    "Sampling5": NearMiss()
}

samples = {}

for name, sampler in sampling_methods.items():
    X_res, y_res = sampler.fit_resample(X, y)
    samples[name] = (X_res, y_res)


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

models = {
    "M1_Logistic": LogisticRegression(max_iter=1000),
    "M2_DecisionTree": DecisionTreeClassifier(),
    "M3_RandomForest": RandomForestClassifier(),
    "M4_KNN": KNeighborsClassifier(),
    "M5_NaiveBayes": GaussianNB()
}


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

results = pd.DataFrame()

for samp_name, (X_s, y_s) in samples.items():
    X_train, X_test, y_train, y_test = train_test_split(
        X_s, y_s, test_size=0.3, random_state=42
    )

    acc_list = []

    for model_name, model in models.items():
        model.fit(X_train, y_train)
        pred = model.predict(X_test)
        acc = accuracy_score(y_test, pred)
        acc_list.append(round(acc*100,2))

    results[samp_name] = acc_list

results.index = list(models.keys())
results


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

,Sampling1,Sampling2,Sampling3,Sampling4,Sampling5
M1_Logistic,33.33,91.92,91.92,92.79,50.00
M2_DecisionTree,50.00,98.69,98.25,97.97,16.67
M3_RandomForest,33.33,100.00,99.34,99.55,16.67
M4_KNN,33.33,98.47,83.84,86.71,83.33
M5_NaiveBayes,33.33,74.67,87.77,82.66,33.33


In [9]:
best_sampling = results.idxmax(axis=1)
best_sampling


,0
M1_Logistic,Sampling4
M2_DecisionTree,Sampling2
M3_RandomForest,Sampling2
M4_KNN,Sampling2
M5_NaiveBayes,Sampling3


In [10]:
results.to_csv("sampling_results.csv")


In [11]:
from google.colab import files
files.download("sampling_results.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
best_sampling = results.idxmax(axis=1)
print(best_sampling)


M1_Logistic        Sampling4
M2_DecisionTree    Sampling2
M3_RandomForest    Sampling2
M4_KNN             Sampling2
M5_NaiveBayes      Sampling3
dtype: object
